读取Excel 内容

20240515 pm excel 批量读取URL后识别线路 并输出

2024年6月11日 22点15分 测试部分链接未识别

* 完善 线路的识别，有识别到sites就认定为线路。
* 增加2代理

### 20240613 pm  "User-Agent": "okhttp/4.1.0" 取代代理链接


In [ ]:
import openpyxl
import requests
import json
import re,os
import warnings 
warnings.filterwarnings("ignore")

def respon_out(respon): #输入respond,判断线路、单仓和多仓
    mcang=re.search("storeHouse",respon,re.M|re.I)
    scang = re.search("urls",respon,re.M|re.I)
    xlu = re.search("sites",respon,re.M|re.I)
    if xlu is not None :
        return "H线路"
    elif xlu is  None and scang is not None and mcang is None:
        return "H单仓"
    elif xlu is  None and scang is  None and mcang is not None:
        return "H多仓"
    else :
        return "other"

def addlink(url): #增加代理链接
    agentlink="https://xn--sss604efuw.com/jm/jiemi.php?url="
    url1=agentlink+url
    return url1

def url_respon(url): #输入原始url,解析网站。返回：0-代理url；1-respone等元组；2-状态+200
    headers={"User-Agent":"okhttp/4.1.0"}
    try:
        respon=requests.get(url,verify=False,timeout =60)
        codema= respon.status_code
        if codema==200:
            conten="首次解析状态:"+str(codema)
            # print(respon.text)
            return url,respon.text,conten
        else:
            respon1=requests.get(url,headers=headers,verify=False,timeout =60)  #更换UA
            codema1= respon1.status_code
            conten1="更换UA状态: "+str(codema1)
            return url,respon1.text,conten1
    except:
        try:
            respon2=requests.get(url=addlink(url),headers=headers,verify=False,timeout =60)  #增加代理2函数
            codema2= respon2.status_code
            conten2="代理网址状态: "+str(codema2)
            return url,respon2.text,conten2
        except:
            return url,"000","解析失败了: "

i=1
# excel_file = r'D:\OneDrive\Learn\pythonLearn\TvBox\tvbox\excel_xianlu.xlsx' #家里路径
excel_file = r'D:\lxd\learn\py\tvbox\excel_xianlu.xlsx' #公司路径
workbook = openpyxl.load_workbook(filename=excel_file)
sheet1= workbook['610']
for row in sheet1.iter_rows(values_only=True):
    if not (row[1]) is None :
        analysis_respone=url_respon(row[1]) 
        strtxt=respon_out(analysis_respone[1]) #返回解析结果,如单仓/多仓/失败
        codetxt=analysis_respone[2]  #返回 url的返回状态 如200，400
        sheet1.cell(row = i,column = 4,value = strtxt)
        sheet1.cell(row = i,column = 5,value = codetxt)
        sheet1.cell(row = i,column = 6,value = analysis_respone[0]) #返回代理链接
        print(strtxt,codetxt,row[1])
    else:
        break
    i=i+1
workbook.save(filename = "excel_0613pm.xlsx")

other 更换UA状态: 404 https://pastebin.com/raw/5NHaxyGR
other 首次解析状态:200 http://肥猫.live
H线路 首次解析状态:200 https://raw.liucn.cc/box/m.json
other 首次解析状态:200 http://小白.love/
H线路 首次解析状态:200 https://szyyds.cn/tv/x.json
H线路 首次解析状态:200 https://github.moeyy.xyz/https://raw.githubusercontent.com/xyq254245/xyqonlinerule/main/XYQTVBox.json
H线路 首次解析状态:200 http://home.jundie.top:81/top98.json
H线路 首次解析状态:200 https://github.moeyy.xyz/https://raw.githubusercontent.com/gaotianliuyun/gao/master/js.json
H线路 首次解析状态:200 https://100km.top/0
H线路 首次解析状态:200 http://cdn.qiaoji8.com/tvbox.json


### 20241120 读取Excel的URL链接 批量解析链接有效性及单多仓


调整解析

In [ ]:
import openpyxl
import requests
import json
import re,os
import warnings 
warnings.filterwarnings("ignore")

def respon_out(respon): #输入respond,判断线路、单仓和多仓
    mcang=re.search("storeHouse",respon,re.M|re.I)
    scang = re.search("urls",respon,re.M|re.I)
    xlu = re.search("sites",respon,re.M|re.I)
    if xlu is not None :
        return "H线路"
    elif xlu is  None and scang is not None and mcang is None:
        return "H单仓"
    elif xlu is  None and scang is  None and mcang is not None:
        return "H多仓"
    else :
        return "other"

def addlink(url): #增加代理链接
    agentlink="https://xn--sss604efuw.com/jm/jiemi.php?url="
    url1=agentlink+url
    return url1

def url_respon(url): #输入原始url,解析网站。返回：0-代理url；1-respone等元组；2-状态+200
    headers={"User-Agent":"okhttp/4.1.0"}
    try:
        respon=requests.get(url,verify=False,timeout =60)
        codema= respon.status_code
        print(codema)
        if codema==200:
            # print(respon.text)
            if respon_out(respon)!='other': #如果解析结果不是other，则返回解析结果
                conten="首次解析状态成功:"+str(codema)
                outcang=respon_out(respon)
                # print(respon.text)
                return url,conten,outcang
            else:
                respon1=requests.get(url,headers=headers,verify=False,timeout =60)  #更换UA
                codema1= respon1.status_code
                conten1="更换UA状态: "+str(codema1)
                outcang1=respon_out(respon1)
                return url,conten1,outcang1
        else:
            respon2=requests.get(url=addlink(url),headers=headers,verify=False,timeout =60)  #增加代理2函数
            codema2= respon.status_code
            conten2="解析状态: "+str(codema2)
            outcang2=respon_out(respon2)
            return url,conten2,outcang2
    except:
        # print("解析失败了!: ",url)
        return url,"000","解析失败了: "

i=1
# excel_file = r'D:\OneDrive\Learn\pythonLearn\TvBox\tvbox\excel_xianlu.xlsx' #家里路径
excel_file = r'D:\lxd\learn\py\tvbox\excel_xianlu.xlsx' #公司路径
workbook = openpyxl.load_workbook(filename=excel_file)
sheet1= workbook['1120']
for row in sheet1.iter_rows(values_only=True):
    if not (row[1]) is None :
        analysis_respone=url_respon(row[1]) 
        strtxt=analysis_respone[2] #返回解析结果,如单仓/多仓/失败
        codetxt=analysis_respone[1]  #返回 url的返回状态 如200，400
        sheet1.cell(row = i,column = 4,value = strtxt)
        sheet1.cell(row = i,column = 5,value = codetxt)
        # sheet1.cell(row = i,column = 6,value = analysis_respone[0]) #返回代理链接
        print(strtxt,codetxt,row[1])
    else:
        break
    i=i+1
workbook.save(filename = "excel_1120_1.xlsx")

### 20241121 ok 
## 从Excel第二列读取链接，判断当前是单仓还是多仓，并把结果写入到Excel文件


In [1]:
import openpyxl
import requests
import json
import re,os
import warnings 
warnings.filterwarnings("ignore")

def respon_out(respon): #输入respond,判断线路、单仓和多仓
    mcang=re.search("storeHouse",respon,re.M|re.I)
    scang = re.search("urls",respon,re.M|re.I)
    xlu = re.search("sites",respon,re.M|re.I)
    if xlu is not None :
        return "H线路"
    elif xlu is  None and scang is not None and mcang is None:
        return "H单仓"
    elif xlu is  None and scang is  None and mcang is not None:
        return "H多仓"
    else :
        return "other"

def addlink(url): #增加代理链接
    agentlink="https://xn--sss604efuw.com/jm/jiemi.php?url="
    url1=agentlink+url
    return url1

def url_respon(url): #输入原始url,解析网站。返回：0-代理url；1-respone等元组；2-状态+200
    headers={"User-Agent":"okhttp/4.1.0"}
    try:
        respon=requests.get(url,verify=False,timeout =60)
        codema= respon.status_code
        if codema==200:
            if respon_out(respon.text)!='other': #如果解析结果不是other，则返回解析结果
                conten="首次解析状态:"+str(codema)
                outcang=respon_out(respon.text)
                # print(respon.text)
                return url,conten,outcang
            else:
                respon1=requests.get(url,headers=headers,verify=False,timeout =60)  #更换UA
                codema1= respon1.status_code
                conten1="更换UA状态: "+str(codema1)
                outcang1=respon_out(respon1.text)
                return url,conten1,outcang1
        else:
            respon2=requests.get(url=addlink(url),headers=headers,verify=False,timeout =60)  #增加代理2函数
            codema2= respon.status_code
            conten2="解析状态: "+str(codema2)
            outcang2=respon_out(respon2.text)
            return url,conten2,outcang2
    except Exception:
        # print("解析失败了!: ",url)
        return url,"失败状态：000","解析失败: "
        
def main():
    i=1
    # excel_file = r'D:\OneDrive\Learn\pythonLearn\TvBox\tvbox\excel_xianlu.xlsx' #家里路径
    excel_file = r'D:\lxd\learn\py\tvbox\excel_xianlu.xlsx' #公司路径
    workbook = openpyxl.load_workbook(filename=excel_file)
    sheet1= workbook['1120']
    for row in sheet1.iter_rows(values_only=True):
        if not row is None :
            analysis_respone=url_respon(row[1]) 
            strtxt=analysis_respone[2] #返回解析结果,如单仓/多仓/失败
            codetxt=analysis_respone[1]  #返回 url的返回状态 如200，400
            sheet1.cell(row = i,column = 4,value = strtxt)
            sheet1.cell(row = i,column = 5,value = codetxt)
            # sheet1.cell(row = i,column = 6,value = analysis_respone[0]) #返回代理链接
            print(strtxt,codetxt,row[1])
        else:
            break
        i=i+1
        workbook.save(filename = "excel_1204.xlsx")
        print("__"*100)
if __name__ == '__main__':
    main()


H单仓 首次解析状态:200 https://tv.youdu.fan:666
________________________________________________________________________________________________________________________________________________________________________________________________________
H线路 首次解析状态:200 https://盒子迷.top/禁止贩卖
________________________________________________________________________________________________________________________________________________________________________________________________________
解析失败:  失败状态：000 https://安卓哥.com
________________________________________________________________________________________________________________________________________________________________________________________________________
other 更换UA状态: 200 http://fongmi.top
________________________________________________________________________________________________________________________________________________________________________________________________________
H线路 首次解析状态:200 http://接口.tvbox.中国
__________________

20250101

In [6]:
import openpyxl
import requests
import json
import re,os
import warnings 
warnings.filterwarnings("ignore")

def respon_out(respon): #输入respond,判断线路、单仓和多仓
    mcang=re.search("storeHouse",respon,re.M|re.I)
    scang = re.search("urls",respon,re.M|re.I)
    xlu = re.search("sites",respon,re.M|re.I)
    if xlu is not None :
        return "H线路"
    elif xlu is  None and scang is not None and mcang is None:
        return "H单仓"
    elif xlu is  None and scang is  None and mcang is not None:
        return "H多仓"
    else :
        return "other"

def addlink(url): #增加代理链接
    agentlink="https://xn--sss604efuw.com/jm/jiemi.php?url="
    url1=agentlink+url
    return url1

def url_respon(url): #输入原始url,解析网站。返回：0-代理url；1-respone等元组；2-状态+200
    headers={"User-Agent":"okhttp/4.1.0"}
    try:
        respon=requests.get(url,verify=False,timeout =60)
        codema= respon.status_code
        if codema==200:
            if respon_out(respon.text)!='other': #如果解析结果不是other，则返回解析结果
                conten="首次解析状态:"+str(codema)
                outcang=respon_out(respon.text)
                # print(respon.text)
                return url,conten,outcang
            else:
                respon1=requests.get(url,headers=headers,verify=False,timeout =60)  #更换UA
                codema1= respon1.status_code
                conten1="更换UA状态: "+str(codema1)
                outcang1=respon_out(respon1.text)
                return url,conten1,outcang1
        else:
            respon2=requests.get(url=addlink(url),headers=headers,verify=False,timeout =60)  #增加代理2函数
            codema2= respon.status_code
            conten2="解析状态: "+str(codema2)
            outcang2=respon_out(respon2.text)
            return url,conten2,outcang2
    except Exception:
        # print("解析失败了!: ",url)
        return url,"失败状态：000","解析失败: "
        
def main():
    i=1
    # excel_file = r'D:\OneDrive\Learn\pythonLearn\TvBox\tvbox\excel_xianlu.xlsx' #家里路径
    excel_file = r'D:\lxd\learn\py\tvbox\excel_xianlu.xlsx' #公司路径
    workbook = openpyxl.load_workbook(filename=excel_file)
    sheet1= workbook['sb']
    for row in sheet1.iter_rows(values_only=True):
        if not row is None :
            analysis_respone=url_respon(row[1]) 
            strtxt=analysis_respone[2] #返回解析结果,如单仓/多仓/失败
            codetxt=analysis_respone[1]  #返回 url的返回状态 如200，400
            sheet1.cell(row = i,column = 4,value = strtxt)
            sheet1.cell(row = i,column = 5,value = codetxt)
            # sheet1.cell(row = i,column = 6,value = analysis_respone[0]) #返回代理链接
            print(strtxt,codetxt,row[1])
        else:
            break
        i=i+1
    workbook.save(filename = "excel_0103sb2.xlsx")
    print("__"*100)
if __name__ == '__main__':
    main()


解析失败:  失败状态：000 http://xmbjm.fh4u.org/ck.txt
解析失败:  失败状态：000 https://raw.gitcode.com/yydg/ggdx/raw/main/Xboxb.json
解析失败:  失败状态：000 http://我不是.肥猫.love:63/接口禁止贩卖
解析失败:  失败状态：000 https://xhdwc.tk/0
解析失败:  失败状态：000 http://8.210.232.168/xc.json
解析失败:  失败状态：000 http://yydsys.top/duo
解析失败:  失败状态：000 http://yydsys.top/duo/ali
解析失败:  失败状态：000 https://jihulab.com/duomv/apps/-/raw/main/fast.json
解析失败:  失败状态：000 http://tv.nxog.top/api.php?mz=xb&id=1&b=欧歌
解析失败:  失败状态：000 http://jk.itvbox.cc:66/可视TV/云星日记/仓库/api.json
解析失败:  失败状态：000 http://52bsj.vip:98/wuaihouse
解析失败:  失败状态：000 http://yydsys.top/duo
解析失败:  失败状态：000 https://la.kstore.space/download/2883/0110.txt
解析失败:  失败状态：000 http://52bsj.vip:98/wuai
解析失败:  失败状态：000 http://itvbox.cc/tvbox/云星日记/1.m3u8
解析失败:  失败状态：000 http://miaotvs.cn/meow
解析失败:  失败状态：000 http://天.陌尘.ic
解析失败:  失败状态：000 http://xmbjm.fh4u.org/ck.txt
解析失败:  失败状态：000 http://肥猫.com/ 
解析失败:  失败状态：000 http://xmbjm.fh4u.org/ck.txt
H线路 首次解析状态:200 https://mirror.ghproxy.com/https://raw.githubu